In [24]:
#Add needed imports
import numpy as np
import pandas as pd
import os
import shap
from numpy import mean
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils import plot_model
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import os
import shap

from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()
#Read data
proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset1.csv')
df = pd.read_csv(train_path)
labels=df['Churn']
x = df.drop(columns=['Churn','Unnamed: 0'],axis = 'columns')
y=np.ravel(labels)
oversample = SMOTENC(categorical_features=[1, 2,17,18,19])
x, y = oversample.fit_resample(x, y)
sc = RobustScaler()
x = pd.DataFrame(sc.fit_transform(x),columns = x.columns)
ii = 1

xgb_model = XGBClassifier(random_state=1,learning_rate=0.05, max_depth=7,eval_metric='mlogloss',use_label_encoder =False ,objective="binary:logistic")
dt_model=DecisionTreeClassifier(random_state=1,criterion='entropy',max_depth = 7,min_samples_leaf=30)  
nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=20, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1                             

Iteration 1


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.105950
                      0.215805
                      0.448174
                      0.222159
                      0.911646
                      0.091948
                      0.260229
                      0.337773
                      0.085859
                      0.136509
                      0.277628
                      0.058794
                      0.054813
                      0.194664
                      0.245985
                      0.059184
                      0.514979
                      0.330723
                      0.262790
                      0.233090
 DT - feature_importance_vals
                     0.000000
                     0.051526
                     0.135379
                     0.056198
                     0.006521
                     0.000287
                     0.161437
                     0.030591
                     0.000000
                     0.026571
                   

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.106569
                      0.240567
                      0.379864
                      0.196034
                      0.819168
                      0.092974
                      0.383107
                      0.331526
                      0.098133
                      0.136216
                      0.267336
                      0.064996
                      0.042589
                      0.212532
                      0.231168
                      0.046790
                      0.507318
                      0.295891
                      0.306688
                      0.200420
 DT - feature_importance_vals
                     0.005118
                     0.054977
                     0.113899
                     0.053657
                     0.045444
                     0.002963
                     0.139187
                     0.006023
                     0.002995
                     0.046534
                   

...

 XGB - feature_importance_vals
                      0.096206
                      0.275793
                      0.380347
                      0.193643
                      0.949091
                      0.086884
                      0.217993
                      0.389914
                      0.091588
                      0.096219
                      0.280931
                      0.065067
                      0.044461
                      0.224436
                      0.244577
                      0.035886
                      0.526119
                      0.280906
                      0.219285
                      0.181251
 DT - feature_importance_vals
                     0.005159
                     0.045345
                     0.103815
                     0.063764
                     0.020077
                     0.000000
                     0.166511
                     0.005042
                     0.001198
                     0.044632
                   

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.110709
                      0.276128
                      0.405510
                      0.176305
                      0.878744
                      0.097974
                      0.320368
                      0.376031
                      0.095080
                      0.111737
                      0.295641
                      0.081982
                      0.025504
                      0.251358
                      0.253310
                      0.017289
                      0.541736
                      0.296416
                      0.194061
                      0.198636
 DT - feature_importance_vals
                     0.000735
                     0.046048
                     0.097867
                     0.058216
                     0.130123
                     0.002379
                     0.057927
                     0.008351
                     0.002895
                     0.052609
                   

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.108222
                      0.263802
                      0.397983
                      0.194238
                      0.710977
                      0.085025
                      0.486814
                      0.459732
                      0.089744
                      0.027036
                      0.278022
                      0.062690
                      0.055612
                      0.209893
                      0.250385
                      0.040083
                      0.530756
                      0.263702
                      0.233668
                      0.195284
 DT - feature_importance_vals
                     0.005909
                     0.046845
                     0.094651
                     0.057825
                     0.033676
                     0.000000
                     0.164609
                     0.011497
                     0.000796
                     0.041803
                   